<div align="center">

# <span style="color: #3498db;">CA2 - Genetic & Game</span>

**<span style="color:rgb(247, 169, 0);">[Student Name]</span> - <span style="color:rgb(143, 95, 195);">[Student Number]</span>**

</div>


<div style="font-family: Arial, sans-serif; line-height: 1.6;">

### 📊 Matplotlib – Data Visualization in Python  

matplotlib is a python library that is mainly used for data visualization. This library allows you to plot different type of figures including scatters and histograms. In the first part of this project you are supposed to implement a genetic algorithm. To visualize plots that are required in the project description use plotting as much as you can because it gives a great insight on what is happening during each run. It also helps you to compare your results whenevever you want to understand effect of different parameters during different runs.
For more information, check [this notebook](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/04.00-Introduction-To-Matplotlib.ipynb) and visit [the website](https://matplotlib.org/stable/tutorials/pyplot.html#sphx-glr-tutorials-pyplot-py).

In [152]:
import matplotlib.pyplot as plt

# <span style="color: #3498db;">Genetic Algorithm</span>

In [153]:
import random
import itertools
import numpy as np

In [154]:
# algorithm parameters
numCoeffs = 41
populationSize = 100
generations = 200
mutationRate = 0.15
functionRange = (-np.pi, np.pi)
sampleCount = 100
delta_huber = 1.0

In [155]:
# These functions are given as samples to use in the algorithm
def getTargetFunction(functionName="sin_cos"):
    def sinCosFunction(t):
        """Target function: sin(2πt) + 0.5*cos(4πt)."""
        return np.sin(2 * np.pi * t) + 0.5 * np.cos(4 * np.pi * t)

    def linearFunction(t):
        """Simple linear function: y = 2t + 1."""
        return 2 * t + 1

    def quadraticFunction(t):
        """Quadratic function: y = 4t^2 - 4t + 2."""
        return 4 * (t**2) - 4 * t + 2

    def cubicFunction(t):
        """Cubic function: y = 8t^3 - 12t^2 + 6t."""
        return 8 * (t**3) - 12 * (t**2) + 6 * t

    def gaussianFunction(t):
        """Gaussian function centered at t=0.5."""
        mu = 0.5
        sigma = 0.1  # Adjust sigma to control the width of the peak
        return np.exp(-((t - mu) ** 2) / (2 * sigma**2))

    def squareWaveFunction(t):
        """Approximation of a square wave. Smoothed for better Fourier approximation."""
        return 0.5 * (np.sign(np.sin(2 * np.pi * t)) + 1)

    def sawtoothFunction(t):
        """Sawtooth wave, normalized to [0, 1]."""
        return (t * 5) % 1

    def complexFourierFunction(t):
        return (
            np.sin(2 * np.pi * t)
            + 0.3 * np.cos(4 * np.pi * t)
            + 0.2 * np.sin(6 * np.pi * t)
            + 0.1 * np.cos(8 * np.pi * t)
        )

    def polynomialFunction(t):
        return 10 * (t**5) - 20 * (t**4) + 15 * (t**3) - 4 * (t**2) + t + 0.5

    functionOptions = {
        "sin_cos": sinCosFunction,
        "linear": linearFunction,
        "quadratic": quadraticFunction,
        "cubic": cubicFunction,
        "gaussian": gaussianFunction,
        "square_wave": squareWaveFunction,
        "sawtooth": sawtoothFunction,
        "complex_fourier": complexFourierFunction,
        "polynomial": polynomialFunction,
    }

    selectedFunction = functionOptions.get(functionName.lower())
    if selectedFunction:
        return selectedFunction
    


In [156]:
# generate samples
tSamples = np.linspace(functionRange[0], functionRange[1], sampleCount)
fSamples = getTargetFunction("polynomial")(tSamples)
A = np.max(np.abs(fSamples)) 
A = A * np.sqrt(2)


<div style="color:rgb(235, 66, 32); font-weight: bold;">⚠️ Important Note:</div>  

Using **NumPy arrays** allows you to perform operations on vectors **more efficiently** and **faster**.

**Avoid using `for` loops** whenever possible, as vectorized operations in NumPy are **optimized for performance** and significantly reduce execution time.  


سه بخش اول

In [157]:
#TODO: Implement the rest of the algorithm

def FiratPopulation():
    population = []
    for _ in range(populationSize):
        coeff = np.random.uniform(-A, A, numCoeffs)
        population.append(coeff.tolist())
    return population


def Fourier(x_array, coeff):
    a0 = coeff[0]
    a_s = np.array(coeff[1:21])
    b_s = np.array(coeff[21:41])

    x_array = np.asarray(x_array) 
    n = np.arange(1, 21).reshape(-1, 1) 

    x_array = x_array.reshape(1, -1)

    cos_terms = np.cos(n * x_array)  
    sin_terms = np.sin(n * x_array)  

    result = a0 / 2 + np.dot(a_s, cos_terms) + np.dot(b_s, sin_terms)
    return result  


def Verification(individual, key):
    ans = Fourier(tSamples, individual)

    ans = np.array(ans)
    if key == "rmse": 
        return np.sqrt(np.mean((ans - fSamples) * (ans - fSamples)))
    elif key == "mae":
        return np.mean(np.abs(ans - fSamples))
    elif key == "r2":
        return 1 - (np.sum((fSamples - ans) ** 2) / np.sum((fSamples - np.mean(fSamples)) ** 2))
    elif key == "huber":
        huber = np.where(np.abs(ans - fSamples) <= delta_huber, 0.5 * (ans - fSamples ** 2), delta_huber * (np.abs(ans - fSamples) - 0.5 * delta_huber))
        return np.mean(huber)
    elif key == "correlation":
        corr = np.corrcoef(ans, fSamples)[0, 1]
        return (1 - corr)


In [158]:
# def GA(): 
#     population = FiratPopulation()
#     good_fitnesses1 = []
#     good_fitnesses2 = []
#     good_fitnesses3 = []

#     for i in range(generations):
#         fitness1 = []
#         fitness2 = []
#         fitness3 = []
#         for j in range(populationSize):
#             fitness1.append(Verification(population[j],"rmse"))
#             fitness2.append(Verification(population[j],"mae"))
#             fitness3.append(Verification(population[j],"r2"))

#         current_best1 = min(fitness1)
#         good_fitnesses1.append(current_best1)  
        
#         current_best2 = min(fitness2)
#         good_fitnesses2.append(current_best2)
        
#         current_best3 = min(fitness3)
#         good_fitnesses3.append(current_best3)
        
#         choose_the_bests = []
#         for j in np.argsort(fitness1)[:int(0.1 * populationSize)].tolist():
#             choose_the_bests.append(population[j])
        
#         new_population = choose_the_bests.copy()
#         while len(new_population) < populationSize:
#             new_population.append(np.random.uniform(-A, A, numCoeffs).tolist())
            
#         population = new_population

#         print(f"generations {i+1}, RMSE: {current_best1:.4f}, MAE: {current_best2:.4f}, R2: {current_best3:.4f}")  
        
#     fitness_scores = []
#     for individual in population:
#         fitness_scores.append(Verification(individual,"rmse"))

#     best_index_individual = np.argmin(fitness_scores)
#     best_individual = population[best_index_individual]
#     return best_individual, good_fitnesses1, good_fitnesses2, good_fitnesses3


بخش 4و5 با انجام اعمال ترکیب و جهش


In [159]:
def Selection(population, fitness):
    size_fight = 10
    selected_parent = []
    for _ in range(2):
        volunteer = np.random.choice(len(population), size_fight, replace=False)
        volunteer_fitness = []
        for v in volunteer:
            volunteer_fitness.append(fitness[v])
        selected_parent.append(population[volunteer[np.argmin(volunteer_fitness)]])
    return selected_parent

def n_point_crossover(parent1, parent2, n=1):
    points = sorted(np.random.choice(range(1, numCoeffs), n, replace=False))
    child1 = []
    child2 = []
    flag = False
    pre = 0
    for point in points:
        if flag:
            child1.extend(parent2[pre:point])
            child2.extend(parent1[pre:point])
        else:
            child1.extend(parent1[pre:point])
            child2.extend(parent2[pre:point])
        flag = not flag
        pre = point
    if flag:
        child1.extend(parent2[pre:])
        child2.extend(parent1[pre:])
    else:
        child1.extend(parent1[pre:])
        child2.extend(parent2[pre:])
    return child1, child2

def Mutate(child):
    for i in range(numCoeffs):
        if np.random.rand() < mutationRate:
            child[i] = np.clip(child[i] + np.random.normal(0, 0.5), -A, A)
    return child


In [160]:

def GA_Powered(): 
    population = FiratPopulation()
    good_fitnesses1 = []
    good_fitnesses2 = []
    good_fitnesses3 = []
    good_fitnesses4 = []
    for i in range(generations):
        fitness1 = []
        fitness2 = []
        fitness3 = []
        fitness4 = []
        for j in range(populationSize):
            fitness1.append(Verification(population[j],"correlation"))
            # fitness2.append(Verification(population[j],"mae"))
            # fitness3.append(Verification(population[j],"rmse"))
            # fitness4.append(Verification(population[j],"huber"))
            

        current_best1 = min(fitness1)  
        good_fitnesses1.append(current_best1) 
        
        # current_best2 = min(fitness2)
        # good_fitnesses2.append(current_best2)
        
        # current_best3 = max(fitness3)
        # good_fitnesses3.append(current_best3)
        
        # current_best4 = max(fitness4)
        # good_fitnesses4.append(current_best4)
        
        choose_the_bests = []
        for j in np.argsort(fitness1)[:int(0.1 * populationSize)].tolist():
            choose_the_bests.append(population[j])
        
        new_population = choose_the_bests.copy()
        while len(new_population) < populationSize:
            parent1, parent2 = Selection(population, fitness1)
            child1 = n_point_crossover(parent1, parent2)[0]
            child2 = n_point_crossover(parent1, parent2)[1]
            child1 = Mutate(child1)
            child2 = Mutate(child2)
            new_population.extend([child1, child2])
        
        population = new_population[:populationSize]
            

        # print(f"generations {i+1}, RMSE: {current_best1:.6f}, MAE: {current_best2:.6f}, R2: {current_best3:.6f}, huber: {current_best4:.6f}")  
        # print(f"generations {i+1}, huber: {current_best4:.6f}")  
        print(f"generations {i+1}, corr: {current_best1:.6f}")  


    fitness_scores = []
    for individual in population:
        fitness_scores.append(Verification(individual,"correlation"))

    best_index_individual = np.argmin(fitness_scores)
    best_individual = population[best_index_individual]
    return best_individual, good_fitnesses1, good_fitnesses2, good_fitnesses3, good_fitnesses4


توابع فیتنس استقاده شده

if key == "rmse": 

        return np.sqrt(np.mean((ans - fSamples) * (ans - fSamples)))

    elif key == "mae":

        return np.mean(np.abs(ans - fSamples))

    elif key == "r2":

        return 1 - (np.sum((fSamples - ans) ** 2) / np.sum((fSamples - np.mean(fSamples)) ** 2))

    elif key == "huber":

        huber = np.where(np.abs(ans - fSamples) <= delta_huber, 0.5 * (ans - fSamples ** 2), delta_huber * (np.abs(ans - fSamples) - 0.5 * delta_huber))
        
        return np.mean(huber)

        corr = np.corrcoef(ans, fSamples)[0, 1]
        
        return (1 - corr)

رسم نمودار

In [ ]:
best_coeffs, history_of_rmse, history_of_mae, history_of_r2, history_of_huber = GA_Powered()
# best_coeffs, history_of_rmse, history_of_mae, history_of_r2 = GA()


ans_fourier = Fourier(tSamples, best_coeffs)  

plt.figure(figsize=(10, 5))
plt.plot(tSamples, fSamples, label="REAL")
plt.plot(tSamples, ans_fourier, "--", label="FOURIER")
plt.show()

final_corr = Verification(best_coeffs, "correlation")
# final_mae = Verification(best_coeffs, "mae")
# final_r2 = Verification(best_coeffs, "r2")
# final_huber = Verification(best_coeffs, "huber")
print(f"final answer: ")
print(f"corr: {final_corr:.6f}")
# print(f"MAE:  {final_mae:.6f}")
# print(f"R²:   {final_r2:.6f}")
# print(f"HUBER:   {final_huber:.6f}")



پاسخ سوالات اخر

1

اگر برای هر ژن اگر n حالت داشته باشیم
و چون 41 ژن داریم n بع توان 41 حالت میتوان برای فضا حالت  متصور بود


2

با افزایش A هم یکی دیگر از راه هایی است که میتوان باعث همگرایی زود تر شون اما ممکن است دقت مون کاهش پیدا کند

 کاهش نرخ جهش:
که تنوع ژنتیکی راکمتر میکند


3

انتخاب تورنومنت 

چند کروموزوم به صورت تصادفی انتخاب میشوند و بین آنها رقابت ایجاد میشود. برنده (با فیتنس بالاتر) به عنوان والد انتخاب میشود

انتخاب چرخ رولت :

	احتمال انتخاب هر کروموزوم متناسب با فیتنس آن است. کروموزومهای با فیتنس بالاتر شانس بیشتری برای انتخاب شدن دارند


4

 افزایش نرخ جهش:
	با افزایش احتمال جهش، ژنهای جدیدی به جمعیت اضافه میشوند که از یکنواختی جلوگیری میکند.

تبدیل کراس اور عادی به ان پوینت کراس اور که باعث تنوع ژن های بیشتری میشود

5

 درصد واریانس متغیر وابسته را که توسط مدل پیشبینی میشود

	در مسائل رگرسیون  برای ارزیابی دقت مدل مفید است.
	در مسائل طبقهبندی یا بهینهسازی غیرخطی (مانند الگوریتم ژنتیک) کاربرد مستقیم 
ندارد، اما میتواند برای ارزیابی کیفیت راهحلها در فاز تحلیل استفاده شود

فرمول

1 - (np.sum((fSamples - ans) ** 2) / np.sum((fSamples - np.mean(fSamples)) ** 2))




### رسم توابع در ورد انجام شده است اما A ها دقیق نبوده و با تغییر A ها میتوان نتایج بهتری بدست اورد

# <span style="color: #3498db;">Minmax Algorithm</span>

In [162]:
import random
import numpy as np
from math import inf
import time
import pygame


In [ ]:
def calculate_block_score(block):
    w = [0, 1, 10, 100, 1000]
    count_red = np.sum(block == 1)
    count_blue = np.sum(block == -1)
    if count_red > 0 and count_blue > 0:
        return 0
    if count_red > 0:
        player = -1 
        count = count_red
    elif count_blue > 0:
        player = 1  
        count = count_blue
    else:
        player = 0  
        count = 0
    score = player * w[count - 1]
    return score

class PentagoGame:
    def __init__(self, ui=False, print=False, depth=2):
        self.board = np.zeros((6, 6), dtype=int)
        self.current_player = 1
        self.ui = ui
        self.depth = depth
        self.nodes_visited = 0
        self.game_over = False
        self.result = None
        self.selected_block = None
        self.move_stage = 0  # 0: place piece, 1: select block, 2: rotate
        self.temp_piece = None
        self.print = print

        if ui:
            pygame.font.init()
            self.screen = pygame.display.set_mode((800, 600))
            pygame.display.set_caption("Pygame Board")
            # self.font = pygame.font.SysFont("Arial", 20)
            self.show_buttons = False
            self.buttons = {
                "rotate_cw": pygame.Rect(650, 200, 100, 50),
                "rotate_ccw": pygame.Rect(650, 300, 100, 50),
            }
            self.setup_controls()
            self.draw_board()

    def setup_controls(self):
        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])   # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.screen.draw_text("CLOCKWISE", self.buttons["rotate_cw"].center)
            self.screen.draw_text("COUNTER-CLOCKWISE", self.buttons["rotate_ccw"].center)

    def hide_rotation_buttons(self):
        self.show_buttons = False

    def show_rotation_buttons(self):
        self.show_buttons = True

    def copy_board(self, board):
        return np.copy(board)

    def rotate_block(self, board, block, direction):
        row_start = (block // 2) * 3
        col_start = (block % 2) * 3
        sub = board[row_start : row_start + 3, col_start : col_start + 3]
        rotated = np.rot90(sub, 3 if direction == "cw" else 1)
        board[row_start : row_start + 3, col_start : col_start + 3] = rotated

    def get_possible_moves(self, board, player):
        moves = []
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    for block in range(4):
                        for dir in ["cw", "ccw"]:
                            moves.append((i, j, block, dir))
        return moves

    def apply_move(self, board, move, player):
        new_board = self.copy_board(board)
        row, col, block, direction = move
        if new_board[row][col] != 0:
            return None
        new_board[row][col] = player
        self.rotate_block(new_board, block, direction)
        return new_board

    def check_winner(self, board):
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    continue

                # Horizontal
                if j <= 1 and np.all(board[i, j : j + 5] == board[i][j]):
                    return board[i][j]

                # Vertical
                if i <= 1 and np.all(board[i : i + 5, j] == board[i][j]):
                    return board[i][j]

                # Diagonal
                if (
                    i <= 1
                    and j <= 1
                    and all(board[i + k][j + k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]

                # Anti-diagonal
                if (
                    i <= 1
                    and j >= 4
                    and all(board[i + k][j - k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]
        if np.all(board != 0):
            return 0
        return None

    def main_hurestic(self, board):
        res = 0
        for i in range(6):
            for j in range(2):
                block = board[i, j:j+5]
                res += calculate_block_score(block)

        for j in range(6):
            for i in range(2):
                block = board[i:i+5, j]
                res += calculate_block_score(block)

        for i in range(2):
            for j in range(2):
                block = []
                for k in range(5):
                    row = i + k
                    col = j + k
                    block.append(board[row][col])
                res += calculate_block_score(block)

        for i in range(2):
            for j in range(4, 6):
                block = []
                for k in range(5):
                    row = i + k
                    col = j - k
                    block.append(board[row][col])
                res += calculate_block_score(block)

        return res

    def hurestic_with_rotation(self, board):
        best = -inf
        for block in range(4):
            for direction in ('cw', 'ccw'):
                b2 = self.copy_board(board)
                self.rotate_block(b2, block, direction)
                val = self.main_hurestic(b2)
                best = max(best, val)
        return best

    def minimax(self, board, depth, prun, maximizing, a=-inf, b=inf):
        self.nodes_visited += 1
        winner = self.check_winner(board)
        if depth == 0 or winner is not None:
            # base = self.main_hurestic(board)
            rot  = self.hurestic_with_rotation(board)
            # return base, None
            return rot, None

        if maximizing:
            value, best_move = -inf, None
            for move in self.get_possible_moves(board, -1):
                new_board = self.apply_move(board, move, -1)
                if new_board is None:
                    continue
                if prun:
                    val, unuseablevalue = self.minimax(new_board, depth-1, prun, False, a, b)
                    a = max(a, val)
                    if b <= a:
                        break
                else:
                    val, unuseablevalue = self.minimax(new_board, depth-1, prun, False)
                if val > value:
                    value, best_move = val, move
            return value, best_move
        else:
            value, best_move = inf, None
            for move in self.get_possible_moves(board, 1):
                new_board = self.apply_move(board, move, 1)
                if new_board is None:
                    continue
                if prun:
                    val, unuseablevalue = self.minimax(new_board, depth-1, prun, True, a, b)
                    b = min(b, val)
                    if b <= a:
                        break
                else:
                    val, unuseablevalue = self.minimax(new_board, depth-1, prun, True)
                if val < value:
                    value, best_move = val, move
            return value, best_move
        
            #TODO: Implement minmax algorithm

    def get_computer_move(self):
        start = time.time()
        self.nodes_visited = 0
        best_value = -inf
        best_move = None

        moves = self.get_possible_moves(self.board, -1)
        for move in moves:
            if self.game_over:
                break
            new_board = self.apply_move(self.board, move, -1)
            if new_board is None:
                continue

            try:
                value, unuseablevalue = self.minimax(new_board,self.depth - 1,prun=False,maximizing=False,a=best_value,b=inf)
                #TODO: Implement alpha-beta pruning algorithm

            except:
                value = -inf

            if value > best_value:
                best_value = value
                best_move = move

        if self.print:
            print(f"Move took {time.time() - start:.2f}s, nodes visited: {self.nodes_visited}")
        self.nodes_visited = 0
        return best_move

    def draw_text(self, text, center_pos, max_width):
        font_size = 24
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, (0, 0, 0))

        text_width = text_surface.get_width()
        if text_width > max_width:
            scale_factor = max_width / text_width
            new_font_size = int(font_size * scale_factor)
            font = pygame.font.Font(None, new_font_size)
            text_surface = font.render(text, True, (0, 0, 0))

        text_rect = text_surface.get_rect(center=center_pos)
        self.screen.blit(text_surface, text_rect)

    def draw_board(self):
        self.screen.fill((0, 0, 0))

        for i in range(6):
            for j in range(6):
                x0 = j * 100
                y0 = i * 100

                if self.board[i][j] == 1:
                    pygame.draw.circle(self.screen, (255, 0, 0), (x0 + 50, y0 + 50), 40)
                elif self.board[i][j] == -1:
                    pygame.draw.circle(self.screen, (0, 0, 255), (x0 + 50, y0 + 50), 40)

                pygame.draw.rect(self.screen, (255, 255, 255), (x0, y0, 100, 100), 1)

        for i in [3, 6]:
            pygame.draw.line(self.screen, (255, 255, 255), (0, i * 100), (600, i * 100), 3)  # Horizontal
            pygame.draw.line(self.screen, (255, 255, 255), (i * 100, 0), (i * 100, 600), 3)  # Vertical

        # Show rotation buttons if in move_stage 2
        if self.move_stage == 2:
            self.highlight_selected_block()
            self.show_rotation_buttons()

        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])  # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text(
                "CLOCKWISE",
                self.buttons["rotate_cw"].center,
                self.buttons["rotate_cw"].width,
            )
            self.draw_text(
                "COUNTER-CLOCKWISE",
                self.buttons["rotate_ccw"].center,
                self.buttons["rotate_ccw"].width,
            )

    def click_handler(self, event):
        if self.game_over or self.current_player != 1:
            return

        x, y = event.pos
        if self.move_stage == 0:  # Place piece
            if x > 600:
                return  # clicks on control area
            col = x // 100
            row = y // 100
            if 0 <= row < 6 and 0 <= col < 6 and self.board[row][col] == 0:
                self.temp_piece = (row, col)
                self.board[row][col] = 1
                self.move_stage = 1
                self.draw_board()

        elif self.move_stage == 1:  # Select block
            if x > 600:
                return
            # which block was clicked
            block_x = 0 if x < 300 else 1
            block_y = 0 if y < 300 else 1
            self.selected_block = block_y * 2 + block_x
            self.move_stage = 2
            self.show_rotation_buttons()
            self.highlight_selected_block()

        elif self.move_stage == 2:  # Rotate
            if self.buttons["rotate_cw"].collidepoint(event.pos):
                self.apply_rotation("cw")
            if self.buttons["rotate_ccw"].collidepoint(event.pos):
                self.apply_rotation("ccw")

    def apply_rotation(self, direction):
        self.rotate_block(self.board, self.selected_block, direction)
        self.current_player = -1
        self.move_stage = 0
        self.selected_block = None
        self.temp_piece = None
        self.hide_rotation_buttons()
        self.draw_board()
        pygame.display.flip()
        self.check_game_over()
        pygame.time.delay(1000)
        self.play_computer_move()

    def highlight_selected_block(self):
        colors = [
            (255, 153, 153),
            (153, 255, 153),
            (153, 153, 255),
            (255, 255, 153),
        ]  # RGB colors

        row_start = (self.selected_block // 2) * 3
        col_start = (self.selected_block % 2) * 3

        pygame.draw.rect(
            self.screen,
            colors[self.selected_block],
            (col_start * 100, row_start * 100, 300, 300),
            5,
        )

    def play_computer_move(self):
        move = self.get_computer_move()
        if move and not self.game_over:
            new_board = self.apply_move(self.board, move, -1)
            if new_board is not None:
                self.board = new_board
                self.current_player = 1
                self.draw_board()
                pygame.display.flip()
                self.check_game_over()
            else:
                print("Invalid computer move!")

    def check_game_over(self):
        winner = self.check_winner(self.board)
        if winner is not None:
            self.game_over = True
            self.result = winner
            print("Game over! Result:", winner)
            if self.ui:
                self.show_game_over_message()

    def show_game_over_message(self):
        self.screen.fill((200, 200, 200))
        pygame.draw.rect(self.screen, (255, 255, 255), (100, 200, 500, 200))
        pygame.draw.rect(self.screen, (0, 0, 0), (100, 200, 500, 200), 3)

        result_text = f"Player {self.result} wins!" if self.result != 0 else "Draw!"
        text_surface = self.font_large.render(result_text, True, (255, 0, 0))
        self.screen.blit(text_surface, (250, 250))

        exit_text = self.font_small.render("Click anywhere to exit", True, (0, 0, 0))
        self.screen.blit(exit_text, (230, 350))
        pygame.display.flip()

    def play(self):
        if self.ui:
            running = True
            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        self.click_handler(event)
                self.draw_board()
                pygame.display.flip()
            pygame.quit()
            return self.result
        else:
            while not self.game_over:
                self.print_board()
                winner = self.check_winner(self.board)
                if winner is not None:
                    return winner

                if self.current_player == 1:
                    move = random.choice(self.get_possible_moves(self.board, 1))
                else:
                    move = self.get_computer_move()

                self.board = self.apply_move(self.board, move, self.current_player)
                self.current_player *= -1
            return self.result

    def print_board(self):
        if self.print == False:
            return
        print("-" * 25)
        for row in self.board:
            print(" ".join(f"{x:2}" for x in row))
        print("-" * 25)
        

      تابع hurestic_with_rotation
هیورستیک با این بخش قوی تر میشود اما نسبت به قبلی زمان اجرا بیشتر میشود

      این بخش ازین کد بدین گونه کار میکند که که علاوه بر نحوه قرار گیری 3تایی 4تایی یا 2 تایی مهره ها ولیو خانه ها را به ازای چرخش به هر دو طرف هر 4 تا باکس حساب میکند که به همین دلیل زمان اجرا بسیار بالا تر میرود اما درصد پیروزی به دلیل دقت بالا بهتر میشود


In [ ]:
if __name__ == "__main__":
    numGames = 5
    numWins, numTies, numLosses = 0, 0, 0
    for i in range(numGames):
        game = PentagoGame(ui=False, print=True, depth=1)  # depth=2 for faster
        result = game.play()
        if result == -1:
            numWins += 1
        elif result == 0:
            numTies += 1
        else:
            numLosses += 1

    print(f"{numWins} wins, {numTies} ties, {numLosses} losses")

سوال 1
 
در فایل های تکست ضمیمه شده نتایج برد بازی ها در 5 بازی برای عمق های 1تا3 مشخص شده است
عمق های بیشتر باعث میشود نود های کمتری دیده شوند زمان اجرا بسیار بالا تر میرود جون اوردر بسیار بیشتر شده است  و شانس پیروی هم با دیدن بیشتر عمق منطقا باید بیشتر شود


سوال 2

ترتیبِ بازدید فرزندان برای بیشترین هرس
بله اگر بتوانیم فرزندان  هر نود را طوری مرتب کنیم که اول بهترین حرکات  پردازش شوند عملاً α–β pruning بیشترین قطع شاخه را خواهد داشت

برای نودهای ماکس حرکات را به ترتیب نزولیِ تابع ارزیابی مرتب می‌کنیم

برای نودهای مسن حرکات را به ترتیب صعودی مرتب می‌کنیم
با این ترتیب α به سرعت افزایش و β به سرعت کاهش می‌یابد و شرط β ≤ α زودتر برقرار می‌شود
اگر نتوانیم هیچ تخمینی از کیفیت حرکت‌ها داشته باشیم این ترتیب‌دهی ممکن نیست و pruning عملاً در بدترین حالت به کارایی Minimax معمولی می‌رسد



سوال 3

Branching Factor  برابر است با تعداد فرزندانِ هر نود در درخت جستجو.

در پنتاگو، در هر وضعیتِ دلخواه تعداد حرکات ممکن = تعداد خانه‌های خالی × ۸ (۴ بلوک × ۲ جهت چرخش) است.

در ابتدای بازی ~۶×۶=۳۶ خانهٔ خالی داریم پس برابر است۲۸۸.

هر چه مهره‌ها پر شوند، خانه‌های خالی کمتر شده و فاکتور به‌تدریج کاهش می‌یابد

سوال 4

α–β pruning همان نتایج Minimax را می‌دهد اما:

به‌محض آنکه یک نود Max متوجه شود مقدار یک فرزند از βِ والد بالاتر است، می‌تواند از بررسی بقیهٔ فرزندان آن Min صرف‌نظر کند.

این «قطع شاخه» باعث می‌شود نیازی به بررسی تمام زیردرخت‌ها نباشد و تعداد نودهای بازدیدشده به‌طور چشمگیری کم شود.



سوال 5

Minimax فرض می‌کند حریف بهینه بازی می‌کند. در حالی که حریف رندم بدون استراتژی عمل می‌کند.

مدل درست برای حریف تصادفی، Expectimax است:

به‌جای نود Min، از نود Chance استفاده می‌کنیم که میانگین (یا وزن‌دار) ارزش فرزندان را می‌گیرد.

در نتیجه ارزش هر نود Chance = ∑‌(احتمال هر حرکت × ارزش فرزند).

Expectimax می‌تواند دقت بهتری برای حریف تصادفی بدهد و نتایج واقعی‌تری تولید کند.